In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    roc_auc_score, f1_score, confusion_matrix
)
from catboost import CatBoostClassifier

warnings.simplefilter('ignore')

def load_data(filename):
    df = pd.read_csv(filename)
    return df

def preprocess_data(df):
    df = df.sample(frac=1, random_state=7).reset_index(drop=True)  # Shuffling
    X = df.drop('target', axis=1)
    y = df['target']
    return X, y

def train_catboost(X_train, y_train, iters, lr, d, a, b):
    cb_model = CatBoostClassifier(
        iterations=iters,
        learning_rate=lr,
        depth=d,
        random_seed=7,
        bagging_temperature=1,
        logging_level='Silent',
        class_weights=[a, b],
        task_type="GPU",
        devices='0'
    )
    cb_model.fit(X_train, y_train)
    return cb_model

def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    cm = confusion_matrix(y_test, preds)
    total = sum(sum(cm))
    roca = round(roc_auc_score(y_test, preds), 4)
    sens = round(cm[1, 1] / (cm[1, 1] + cm[0, 1]), 4)
    spec = round(cm[0, 0] / (cm[1, 0] + cm[0, 0]), 4)
    f1sc = round(f1_score(y_test, preds), 4)
    return spec, sens, roca, f1sc

def model_cats_test0(iters0, lr0, d0, a0, b0, train_name0, test_name0):
    # Load and preprocess training data
    train_df = load_data(train_name0)
    X_train, y_train = preprocess_data(train_df)

    # Load and preprocess testing data
    test_df = load_data(test_name0)
    X_test, y_test = preprocess_data(test_df)

    # Train CatBoost
    cb_model = train_catboost(X_train, y_train, iters0, lr0, d0, a0, b0)

    # Evaluate model
    spec, sens, roca, f1sc = evaluate_model(cb_model, X_test, y_test)

    return spec, sens, roca, f1sc


In [ ]:
tr = "TrainSet-80-44754_3374_TargetIn.csv"
ts = "TestSet-20-44754_3374_TargetIn.csv"
model_cats_test0(10, 0.3, 3, 0.50, 0.50, tr, ts)